In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [22]:
url = "https://github.com/swarj/MLAssignment3/blob/main/usnewshealth.xlsx?raw=true"
raw_df = pd.read_excel(url)
raw_df.columns = ['Tweet Content']
#s1 = pd.Series([])
my_list = []
#df.style
newdf = raw_df['Tweet Content'].str.split('|').str[2]
#df['Tweet Content'] = df['Tweet Content'].re.sub('|').str[2]
#newdf
newdf.columns = ['Tweets']
for indx in newdf:
  indx = re.sub('@[\w]+','', indx)
  indx = re.sub("#","", indx)
  indx = re.sub(r'http\S+', '', indx)
  indx = indx.lower()
  my_list.append(indx)
  #print(indx)
  #newdf[indx] = indx
  #print(newdf[indx])

# pre-processed dataframe
df = pd.DataFrame(my_list)
# for i in range(len(my_list)):
#   print (my_list[i])
#   # print ('\n')
df.style

,0
0,rt : any dads out their who struggled w/ depression or anxiety after their kid was born? let's talk! amiller[at]usnews[dot]co…
1,america's problem with diabetes in one map: by
2,think water & fiber will cure your constipation? unlikely. here's why:
3,about to lose it? heretry one of these office-approved relaxation exercises: stress
4,should you get your baby's dna decoded? genetics
5,3 easy ways to eliminate work stress: via
6,7 steps for choosing & keeping the right personal trainer: by personaltraining
7,"getting ready for bikini season? don't. with the ""anti-bikini season diet"""
8,there's a reason you're still fat. in fact6 of them. weightloss
9,are your allergies just annoying -- or do they require medical attention? how to know:
